In [1]:
## Importing packages

install.packages("MASS")
install.packages("Matrix")
install.packages("lme4")
install.packages("tableone")
install.packages("Matching")
install.packages("MatchIt")

################

library(tidyverse) # metapackage with lots of helpful functions
library(MatchIt)
library(tableone)
library(Matching)

#################
list.files(path = "../input")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1.9000     ✔ purrr   0.3.2     
✔ tibble  2.1.3          ✔ dplyr   0.8.3     
✔ tidyr   0.8.3          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: MASS


Attaching package: ‘MASS’


The following o

character(0)

In [5]:
library(Matching)
load(lalonde)

ERROR: Error in load(lalonde): bad 'file' argument


## Description of Data
#### Number of Subjects: 614
#### Treatment is treat: It is equal to 1 if the subject received labor training and equal to 0 otherwise
#### Outcome is re78: Is the post intervention outcome, income in 1978
#### Potential Confoundings: age, educ, black, hispan, married, nodegree, re74, re75

## Create a Data Frame with Counfoundings

In [16]:
AGE<-as.numeric(lalonde$age)
EDUC<-as.numeric(lalonde$educ)
BLACK<-as.numeric(lalonde$black==1)
HISP<-as.numeric(lalonde$hispan==1)
MARRIED<-as.numeric(lalonde$married==1)
NODEGREE<-as.numeric(lalonde$nodegree==1)
RE74<-as.numeric(lalonde$re74)
RE75<-as.numeric(lalonde$re75)
Treatment<-as.numeric(lalonde$treat)
Outcome<-as.numeric(lalonde$re78)

In [55]:
################## COUNFOUNDING VARS
xvars=c("AGE","EDUC","BLACK","HISP","MARRIED","NODEGREE","RE74","RE75","Outcome")
################## 
mydata<-cbind(AGE,EDUC,BLACK,HISP,MARRIED,NODEGREE,RE74,RE75,Treatment,Outcome)
mydata=data.frame(mydata)

## Stratify Data Base on Treatment & Report the SMD

In [56]:
table1<-CreateTableOne(vars=xvars,strata="Treatment",
                      data=mydata,test=FALSE)
print(table1,smd=TRUE)

                      Stratified by Treatment
                       0                 1                 SMD   
  n                        429               185                 
  AGE (mean (SD))        28.03 (10.79)     25.82 (7.16)     0.242
  EDUC (mean (SD))       10.24 (2.86)      10.35 (2.01)     0.045
  BLACK (mean (SD))       0.20 (0.40)       0.84 (0.36)     1.668
  HISP (mean (SD))        0.14 (0.35)       0.06 (0.24)     0.277
  MARRIED (mean (SD))     0.51 (0.50)       0.19 (0.39)     0.719
  NODEGREE (mean (SD))    0.60 (0.49)       0.71 (0.46)     0.235
  RE74 (mean (SD))     5619.24 (6788.75) 2095.57 (4886.62)  0.596
  RE75 (mean (SD))     2466.48 (3292.00) 1532.06 (3219.25)  0.287
  Outcome (mean (SD))  6984.17 (7294.16) 6349.14 (7867.40)  0.084


## Fit a Propensity Score Model (Using Logistic Regression)


In [57]:
psmodel<-glm(Treatment~AGE+EDUC+BLACK+HISP+
             MARRIED+NODEGREE+RE74+RE75,
    family=binomial(),data=mydata)

In [62]:
pscore<-psmodel$fitted.values
set.seed(931139)
psmatch<-Match(Tr=mydata$Treatment,M=1,X=pscore,replace=FALSE,caliper=0.1)
matched_propensity<-mydata[unlist(psmatch[c("index.treated","index.control")]),]

In [63]:
matchedtable_propensity<-CreateTableOne(vars=xvars,strata="Treatment",data=matched_propensity,test=FALSE);
print(matchedtable_propensity,smd=TRUE)

                      Stratified by Treatment
                       0                 1                 SMD   
  n                        111               111                 
  AGE (mean (SD))        26.27 (11.10)     26.22 (7.18)     0.006
  EDUC (mean (SD))       10.37 (2.66)      10.25 (2.31)     0.047
  BLACK (mean (SD))       0.72 (0.45)       0.74 (0.44)     0.040
  HISP (mean (SD))        0.11 (0.31)       0.10 (0.30)     0.029
  MARRIED (mean (SD))     0.24 (0.43)       0.24 (0.43)    <0.001
  NODEGREE (mean (SD))    0.66 (0.48)       0.65 (0.48)     0.019
  RE74 (mean (SD))     2704.56 (4759.89) 2250.49 (5746.14)  0.086
  RE75 (mean (SD))     1969.10 (3169.08) 1222.25 (3081.19)  0.239
  Outcome (mean (SD))  4904.37 (5832.77) 6151.18 (6710.66)  0.198


## Outcome Analysis--Paired T-test & Mcnemar Test

In [65]:
y_trt_propensity=matched_propensity$Outcome[matched_propensity$Treatment==1];
y_con_propensity=matched_propensity$Outcome[matched_propensity$Treatment==0]

In [67]:
diffy_propensity<-y_trt_propensity-y_con_propensity;
t.test(diffy_propensity)


	One Sample t-test

data:  diffy_propensity
t = 1.4824, df = 110, p-value = 0.1411
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -420.0273 2913.6398
sample estimates:
mean of x 
 1246.806 
